# Rickie's Sweater Pattern Generator

This is a Python knitting pattern generator. It creates patterns for classic knit sweaters with a loose fit, which are based on inputted measurements and style choices. It accepts chest circumference measurements from 24 in to 67 in and torso length measurements from 16 in to 29 in.

### Imports

The packages ipywidgets, interact_manual, numpy, and math must be imported for this pattern generator code to work.

In [68]:
import ipywidgets as widgets
from ipywidgets import interact_manual
import numpy as np
import math

### Size Dictionaries

In garment creation there are 2 kinds of sizing: straight sizing and fit sizing. When something is straight sized, that means that every size option for a finished piece will have the same proportions, just at different scales. This type of sizing is standard in the fashion industry and the knitting community because it is simplest to write patterns for. However, this sizing does not reflect the actual proportions of a human body. This generator uses fit sizing, but it began as a straight sized pattern I wrote.

That pattern has 11 sizes and each of those sizes have corresponding finished chest and torso dimensions. For my fit sized pattern, I may mix and match the instructions for a chest circumference of one size with the instructions of a torso length for another. Below are a series of dictionaries which hold the values for each size and their finished measurements. This has been included for easy referencing of the straight size finished measurement values.

In [69]:
one_dict = {"finished chest":32, "finished torso":25.5, "size": 1}
two_dict = {"finished chest":36, "finished torso":26, "size": 2}
three_dict = {"finished chest":40, "finished torso":26.5, "size": 3}
four_dict = {"finished chest":44, "finished torso":27, "size": 4}
five_dict = {"finished chest":48, "finished torso":27.5, "size": 5}
six_dict = {"finished chest":52, "finished torso":28, "size": 6}
seven_dict = {"finished chest":56, "finished torso":28.5, "size": 7}
eight_dict = {"finished chest":60, "finished torso":29, "size": 8}
nine_dict = {"finished chest":64, "finished torso":29.5, "size": 9}
ten_dict = {"finished chest":68, "finished torso":30, "size": 10}
eleven_dict = {"finished chest":72, "finished torso":30.5, "size": 11}

### Sizing Functions

The chest and torso sizes for this function are discrete, rather than continuous variables. However, the users are free to input their measurements up to a hundredth of accuracy. These sizing functions match a user’s measurement to the most appropriate chest and torso size. 

The function works by iterating over the thresholds list. For each threshold, it checks if the given torso_length is less than the threshold value.
Once a threshold is found that the torso_length is less than, the corresponding dictionary is selected from the dictionaries list.
The function then returns the "finished torso" value from the selected dictionary, which represents the final torso length of the garment.

In [70]:
def chest_sizing_output(chest_circumference):
    thresholds = [28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 67]
    dictionaries = [one_dict, two_dict, three_dict, four_dict, five_dict, six_dict, 
                   seven_dict, eight_dict, nine_dict, ten_dict, eleven_dict]
    
    for i, threshold in enumerate(thresholds):
        if chest_circumference < threshold:
            garment_chest_measurement = dictionaries[i]["finished chest"]
            return garment_chest_measurement

def torso_sizing_output(torso_length):
    thresholds = [21.6, 22.1, 22.6, 23.1, 23.6, 24.1, 24.6, 25.1, 25.6, 26.1, 29]
    dictionaries = [one_dict, two_dict, three_dict, four_dict, five_dict, six_dict, 
                   seven_dict, eight_dict, nine_dict, ten_dict, eleven_dict]
    
    for i, threshold in enumerate(thresholds):
        if torso_length < threshold:
            garment_torso_measurement = dictionaries[i]["finished torso"]
            return garment_torso_measurement

#### Back Panel Cast On Function

This function calculates the number of stitches needed to cast on for the back of the garment.

In [71]:
def back_cast_on_st_ct(chest_sizing_var):
    n = (chest_sizing_var - 32) / 4
    back_co_ct = 81 + (10 * n)
    return (back_co_ct)

#### Back Panel First Bind Off Stitches

This function returns the number of stitches the knitter will need to cast off when they reach the point where they need to start shaping the armholes for the back panel. It uses a dictionary which links completed chest measurements to numbers of stitches to bind off, and it returns the smallest value (2) as the default.

In [72]:
def get_bind_off_stitches(garment_chest_measurement):
    bind_off_dict = {
        32: 2,
        36: 3,
        40: 4,
        44: 5,
        48: 6,
        52: 7,
        56: 8,
        60: 9,
        64: 10,
        68: 11,
        72: 12
    }
    
    return bind_off_dict.get(garment_chest_measurement, 2)

#### Back Panel Stitch Count After First Bind Off

This function calculates the number of stitches the back panel will have immediately after the first block of decreases (bind offs). 

In [73]:
def get_stitch_count_after_bind_off(cast_on_stitches, bind_off_stitches):
    return cast_on_stitches - (2 * bind_off_stitches)

#### Back Panel Main Decrease Section Repeat Count

This function returns the number of times the knitter will need to repeat a decrease row. It uses a dictionary which links completed chest measurements to numbers of repeats make, and it returns the smallest value (1) as the default.

In [74]:
def determine_repeat_count(garment_chest_measurement):
    repeat_count_dict = {
        32: 1,   
        36: 3,   
        40: 4,   
        44: 6,   
        48: 7,   
        52: 9,    
        56: 10,  
        60: 12, 
        64: 13,  
        68: 15,  
        72: 16    
    }
    return repeat_count_dict.get(garment_chest_measurement, 1)

#### Stitches Left After the Main Decrease Section

This function calculates the number of stitches the back panel will have following the main section of decreases.

In [75]:
def remaining_stitches_after_decreases(stitch_count_after_bind_off, repeat_count):
    return stitch_count_after_bind_off - (2 * repeat_count)

#### Armhole Length After Working Even

This function returns the length that the armholes will measure after working even across the decreased section. It uses the dictionary armhole_length_dict to map torso measurements to known armhole lengths. It returns the smallest value (8.5) as the default.

In [76]:
def armhole_length_after_working_even(garment_torso_measurement):
    armhole_length_dict = {
        25.5: 8.5, 26: 9, 26.6: 9.5, 27: 10, 27.5: 10.5, 28: 11, 28.5: 11.5,
        29: 12, 29.5: 12.5, 30: 13, 30.5: 13.5
    }
    
    for threshold in sorted(armhole_length_dict.keys(), reverse=True):
        if garment_torso_measurement >= threshold:
            return armhole_length_dict[threshold]
    return 8.5

#### First Bind Off of the Back Shoulder Section

This function returns the number of stitches to bind off at the beginning of shaping the shoulder. It uses the dictionary first_decrease_dict to map chest measurements to decrease values. It returns the smallest value (5) as the default.

In [77]:
def first_shoulder_bind_off_stitches(garment_chest_measurement):
    first_decrease_dict = {
        32: 5,   
        36: 6,   
        40: 8,   
        44: 9,   
        48: 9,   
        52: 10,  
        56: 10,  
        60: 11,  
        64: 11,  
        68: 13,  
        72: 14   
    }
    
    return first_decrease_dict.get(garment_chest_measurement, 5)

#### Second Bind Off of the Back Shoulder Section 

This function calculates how many stitches the knitter should bind off during theri second decrease of the shoulder section. 

In [78]:
def second_shoulder_bind_off_stitches(remaining_stitches, first_decrease_amount):
    second_decrease_amount = ((remaining_stitches - 2 * first_decrease_amount - 41) / 4)
    return int(second_decrease_amount)

#### Length of Front Panel Armholes

This function calculates the length that the armholes on the panel will measure

In [79]:
def front_armhole_length(armhole_length):
    return armhole_length - 2

#### Stitches Remaining on Either Side of the Markers

This function calculates the number of stitches which will be left on either side of the markers when the knitter begins shaping the neck on the front panel.

In [80]:
def stitches_on_each_side_of_markers(remaining_stitches):
    return (remaining_stitches - 29) // 2

#### Stitches Remaining After Front Shoulder Section Decreases

This function calculates how many stitches the shoulder section will have after the decreases on that section.

In [81]:
def stitches_after_decreases(side_stitches, num_decreases=6):
    remaining_side_stitches = side_stitches - num_decreases
    return remaining_side_stitches

#### Sleeve Cast On Number

This function returns the number of stitches to cast on for the sleeves. It is based off of the torso measurement, as arm length is more closely correlated to that value than to a chest measurement. 

It only has 2 possible values, becasue sleeves are a more universal shape across bodies.

In [82]:
def sleeve_cast_on_amount(garment_torso_measurement):
    if garment_torso_measurement < 27.5:
        return 45
    else:
        return 49

#### Determining Sleeve Increase Rows

The function is determining a set of increases and increments for the sweater based on the finished garment torso measurement. It does this by:
Mapping the torso measurement to an index (after subtracting 24). It uses that index to retrieve values from 5 predefined lists of intervals, and returns the corresponding values as a tuple.

There are fewer options for the values on the sleeves, becasue of the afformentioned universality across sleeve shapes. The 4 largest sizes and the 2 smallest sizes fit most bodies better, and so continung to increase or decrease size does *not* lead to a better fit.


In [83]:
def determine_inc_rows(garment_torso_measurement):
    intervals_set1 = [10, 10, 8, 6, 6, 6, 6, 4, 4, 4, 4]
    intervals_set2 = [3, 3, 5, 15, 13, 9, 3, 24, 24, 24, 24]
    intervals_set3 = [8, 8, 6, 4, 4, 4, 4, 0, 0, 0, 0]
    intervals_set4 = [8, 8, 9, 1, 4, 10, 19, 0, 0, 0, 0]
    intervals_set5 = [69, 69, 75, 79, 85, 89, 95, 99, 99, 99, 99]
    
    size_index = round(garment_torso_measurement) - 24
    size_index = max(0, min(size_index, len(intervals_set1) - 1))
    
    return (
        intervals_set1[size_index],  
        intervals_set2[size_index],  
        intervals_set3[size_index], 
        intervals_set4[size_index], 
        intervals_set5[size_index] 
    )

#### Bind Off For Shoulder Cap Shaping

This function maps the torso measurement to an index (after subtracting 24), retrieves the corresponding value from the bind_off_stitches list at that index, and returns the number of stitches to bind off.

In [84]:
def determine_bind_off_shape_cap(garment_torso_measurement):
    bind_off_stitches = [3, 3, 4, 5, 6, 7, 8, 9, 9, 9, 9]
    size_index = round(garment_torso_measurement) - 24
    size_index = max(0, min(size_index, len(bind_off_stitches) - 1))
    bind_off = bind_off_stitches[size_index]
    return bind_off

#### Stitches Remaining After the First Bind Off
This function maps the torso measurement to an index (after subtracting 24), retrieves the corresponding value from the resulting_stitches list at that index, and returns the number of stitches left after the bind off.

In [85]:
def determine_sts_shape_cap(garment_torso_measurement):
    resulting_stitches = [63, 63, 67, 69, 73, 75, 79, 81, 81, 81, 81]
    size_index = round(garment_torso_measurement) - 24
    size_index = max(0, min(size_index, len(resulting_stitches) - 1))
    shape_cap_stitches = resulting_stitches[size_index]
    return shape_cap_stitches

#### Final Shoulder Cap Shaping Values

This function determines the number of final decrease rows to repeat, the final number of stitches remaining after decreases, and the number of stitches that still need to be bound off.

It uses the garment's torso measurement (rounded and adjusted) to look up corresponding values in predefined lists.

In [86]:
def final_shape_cap_dec_and_bindoff(garment_torso_measurement):
    repeat_decrease_row = [24, 24, 26, 26, 28, 28, 29, 29, 29, 29, 29]
    resulting_stitches_after_decrease = [13, 13, 13, 15, 15, 17, 19, 21, 21, 21, 21]
    remaining_stitches_to_bind_off = [9, 9, 9, 11, 11, 13, 15, 17, 17, 17, 17]
    size_index = round(garment_torso_measurement) - 24
    size_index = max(0, min(size_index, len(repeat_decrease_row) - 1))
    cap_repeats = repeat_decrease_row[size_index]
    cap_final_stitches = resulting_stitches_after_decrease[size_index]
    cap_remaining_bind_off = remaining_stitches_to_bind_off[size_index]
    return cap_repeats, cap_final_stitches, cap_remaining_bind_off

#### Skeins of Yarn Needed

This function returns the skeins of Red Heart Super Saver yarn required for different finsihed chest sizes, because chest sizes map better to overall size than torso length. The values have been determined by practical experience.

In [87]:
def skeins_of_read_heart_ss(garment_chest_measurement):
    skeins = {
        32: 4,   
        36: 4,   
        40: 5,   
        44: 5,   
        48: 5,   
        52: 5,  
        56: 6,  
        60: 6,  
        64: 7,  
        68: 7,  
        72: 8   
    }
    
    return skeins.get(garment_chest_measurement, 4)

### Running the Knitting Pattern

The function below contains code for 3 main parts: 
* Calling the functions designed above
* The written text of the pattern
* the interactive widget

The first section calls the functions which were defined in the above code block. Any stitch counts or decrease values (variables which don't make sense as ints) which were not already ints were converted to such at this step.

The second section is the largest, and it is the actual text of the pattern set up in a series of print functions. It is an adaptation and an expansion of a pattern I already had written. Typically, when a pattern with multiple sizes reaches a step where the instructions vary across sizes, the text will look like this:

*Bind off 3(4, 5, 6, 7) stitches at the beginning of the next 4 rows*

When using that format, the knitter has to remember where in the list the number for their size is. In my generator, every instance where such a list like, the program inserts the knitters individual value which came from their measurements. This was done using formatted string literals. 

If statements have been inserted into this portion of the code to allow for variation in collar and cuff style.

The final section uses the ipywidgets package, and it essentially sets up a user interface within the notebook. It codes for the widget where the knitter inputs their measurements and style preferences into float text boxes and dropdown menus respectively. The text box for chest accepts values from 24 to 67, and the text box for torso accepts values from 16 to 29, both on a 0.1 step. The default values for chest and torso are set to the measurements of the coder. 

The collar and cuffs dropdown boxes both have the same options: 1x1, Rolled, and Folded. The default for both is 1x1, as that is what I initially wrote in the pattern. 

The very end of the block is the Interact Manual statement, which makes the whole function run and generate the pattern upon the user clicking the *Run Interact* button.




In [88]:
def run_knitting_design(collar, cuffs, chest_circumference, torso_length):

    garment_chest_measurement = chest_sizing_output(chest_circumference)
    
    garment_torso_measurement = torso_sizing_output(torso_length)
    
    cast_on_stitches = int(back_cast_on_st_ct(garment_chest_measurement))

    bind_off_stitches = int(get_bind_off_stitches(garment_chest_measurement))
    
    stitch_count_after_bind_off = int(get_stitch_count_after_bind_off(cast_on_stitches, bind_off_stitches))

    repeat_count = int(determine_repeat_count(garment_chest_measurement))
    
    remaining_stitches = int(remaining_stitches_after_decreases(stitch_count_after_bind_off, repeat_count))
    
    armhole_length = armhole_length_after_working_even(garment_torso_measurement)
    
    first_decrease_amount = int(first_shoulder_bind_off_stitches(garment_chest_measurement))
    
    remaining_stitches_after_first = int(cast_on_stitches - 2 * first_decrease_amount)
    
    second_decrease_amount = int(second_shoulder_bind_off_stitches(remaining_stitches_after_first, first_decrease_amount))
    
    front_armhole = front_armhole_length(armhole_length)

    side_stitches = int(stitches_on_each_side_of_markers(remaining_stitches))
    
    final_side_stitches = int(stitches_after_decreases(side_stitches, num_decreases=6))
    
    sleeve_cast_on = sleeve_cast_on_amount(garment_torso_measurement)
    
    shape_cap_bind_off = determine_bind_off_shape_cap(garment_torso_measurement)
    
    shape_cap_stitches = int(determine_sts_shape_cap(garment_torso_measurement))

    sleeve_dec_every_what_row_1, sleeve_how_many_times_1, sleeve_dec_every_what_row_2, sleeve_how_many_times_2, sleeve_how_many_stitches = determine_inc_rows(garment_torso_measurement)
    
    cap_repeats, cap_final_stitches, cap_remaining_bind_off = final_shape_cap_dec_and_bindoff(garment_torso_measurement)
    
    skeins = skeins_of_read_heart_ss(garment_chest_measurement)
    
    print("This pattern has been made to your measurements, happy knitting!")
    print( )
    print("Gauge:")
    print("20 stitches and 27 rows = 4 inches sq on larger needles")
    print("PLEASE KNIT TENSION SWATCH TO ENSURE PROPER FIT")
    print( )
    print("Supplies")
    print(f"Yarn: {skeins} skeins of Red Heart Super Saver")
    print("Needles: 5mm circular knitting needles and 4mm circular knitting needles, OR needles required to obtain gauge")
    print("Notions: 2 stitch markers, 2 stitch holders, 1 tapestry needle, scissors")
    print( )
    print( )
    print( )
    print("Instructions Begin:")
    print( )
    print( )
    print("Back Panel") 
    print( )
    print( )
    print(f"With the smaller knitting needles, cast on {cast_on_stitches} stitches")
    print( )
    print("Ribbing Row 1: (Right Side): k1, * p1, k1, rep from * across.")
    print("Ribbing Row 2: p1, * k1, p1, rep from * across.")
    print( )
    print("Repeat these 2 rows until your piece measures 3 inches from the beginning, end on a Wrong Side")
    print( )
    print("Change to your larger needles and work even in stockinette stitch until the piece measures 17in from the beginning, end on a Wrong Side.")
    print( )
    print(f"Shape Armholes: Bind off {bind_off_stitches} stitches the the beginning of the next 2 rows --- {stitch_count_after_bind_off} stitches")
    print("Decrease Row (Right Side): k1, k2tog, knit to last 3 sts, SSK, k1 -— 2 stitches decreased.")
    print( )
    print(f"Repeat the decrease row every Ride Side row {repeat_count} times more --- {remaining_stitches} stitches")
    print( )
    print(f"Work even until the armhole measures {armhole_length} inches. End on a Wrong Side.")
    print( )
    print(f"Shape Shoulders: Bind off {first_decrease_amount} stitches at the beginning of the next 2 rows, then {second_decrease_amount} stitches")
    print("at the beginning of the following 4 rows.")
    print( )
    print("At this point, we are finished with the back panel of the sweater. Slip the remaining 41 stitches onto a stitch holder")
    print("or piece of scrap yarn, and secure it for later")
    print( )
    print( )
    print( )
    print("Front Panel")
    print( )
    print( )
    print(f"The Front panel is worked the same as the back panel until the piece measures {front_armhole} inches,") 
    print(f"end on a Wrong Side --- {remaining_stitches} stitches")
    print( )
    print(f"Mark the center 29 stitches on the last row you worked, with {side_stitches} stitches remaining on either side of your markers")
    print( )
    print(f"Shape Neck (Right Side): On the next row, knit to your first marker. Now slip the {side_stitches} stitches you just")
    print(f"knit onto a stitch holder or piece of scrap yarn, and secure it for later.")
    print( )
    print(f"Bind off the 29 center stitches and continue knitting to the end --- {side_stitches} stitches.")
    print("Wrong Side: p across")
    print(f"Decrease Row (Right Side): k1, k2tog, knit to end -— 1 stitch decreased.")
    print( )
    print(f"Repeat decrease row 5 times more --- {final_side_stitches} stitches.")
    print( )
    print(f"Work even if necessary until armhole measures {armhole_length} inches, end on a Right Side.")
    print( )
    print(f"Shape Right Shoulder: Bind off {first_decrease_amount} stitches at the beginning of the 1st Wrong Side row.")
    print(f"Next, bind off {second_decrease_amount} stitches at the beginning of the next 2 Wrong Side rows.")
    print("Fasten off.")
    print( )
    print( )
    print(f"Left Side: With Wrong Side facing out, slip your {side_stitches} stitches from the holder onto the larger needles.")
    print("Join yarn and purl 1 Wrong Side row.")
    print(f"Decrease Row (Right Side):  Knit to the last 3 sts, SSK, k1 -— 1 stitch decreased.")
    print( )
    print(f"Repeat decrease row every right side 5 times more --- {final_side_stitches} stitches.")
    print( )
    print(f"Work even if necessary until armhole measures {armhole_length} inches")    
    print( )  
    print(f"Shape left Shoulder: Bind off {first_decrease_amount} stitches at the beginning of the first Wrong Side row.")
    print(f"Next, bind off {second_decrease_amount} stitches at the beginning of the next 2 Wrong Side rows.")
    print("Fasten off.")
    print( )
    print( )
    print( )
    print("Sleeves")
    print( )
    print( )
    if cuffs == '1x1 rib':  
        print(f"With your smaller knitting needles, cast on {sleeve_cast_on} stitches.")
        print( )
        print("Work in 1x1 ribbing as for the back for 3 inches, end on a Wrong Side row.")
        print( )
        print("Change to your larger needles and work even in stockinette stitch until the sleeve measures 4 inches from the")
        print("beginning, end on a Wrong Side.")
    elif cuffs == 'Folded':
        print(f"With your smaller knitting needles, cast on {sleeve_cast_on} stitches.")
        print( )
        print("Work in 1x1 ribbing as for the back for 6 inches, end on a Wrong Side row.")
        print( )
        print("Once you have 6 inches of ribbing, flip up only the bottom of your work so that the Wrong Side is flush with")
        print("the Right Side that you are knitting on. Next, pick up and knit the corresponding bottom stitch (from your cast on).")
        print("This is your next row. Instead of knitting just your 1x1 ribbing stitches you just made, you are knitting those")
        print("stitches PLUS your first row stitches together. This forms the 'fold' of your folded cuff.")
        print("purl one more row across (Wrong Side).")
        print( )
        print("Change to your larger needles and work even in stockinette stitch until the sleeve measures 4 inches from the")
        print("beginning, end on a Wrong Side.")
    else: 
        print(f"With your larger knitting needles, cast on {sleeve_cast_on} stitches.")
        print( )
        print("work in stockinette for 3 inches, end on a Wrong Side row")
    print( )
    print(f"Increase Row (Right Side): k1, M1R, k to last st, M1L, k1 -— 2 stitchs increased.")
    print( )
    print(f"Repeat increase row every { sleeve_dec_every_what_row_1}th row {sleeve_how_many_times_1} times, then every {sleeve_dec_every_what_row_2}th row {sleeve_how_many_times_2}")
    print(f"times --- {sleeve_how_many_stitches} stitches.")
    print( )
    print("Work even until the sleeve measures 19 inches from the beginning, end on a Wrong Side.")
    print( )
    print(f"Shape Cap: Bind off {shape_cap_bind_off} stitches at the beginning of the nest 2 rows --- {shape_cap_stitches} stitches.")
    print( )
    print(f"Decrease Row (Right Side):  k1, k2tog, k to last 3 sts, SSK, k1 -— 2 stitches decreased.")
    print( )
    print(f"Repeat decrease row every Right Side row {cap_repeats} times more, end on a Wrong Side --- {cap_final_stitches} stitches.")
    print( )
    print("Bind off 2 stitches at the beginning of the next 2 rows.")
    print( )
    print(f"Bind off the remaining {cap_remaining_bind_off} stitches.")
    print( )
    print( )
    print("Make 2")
    print( )
    print( )
    print( )
    print("Finishing")
    print( )
    print( )
    print("Sew the shoulder seams of front and back panels")
    print( )
    print("Collar")
    print( )
    print("Start with the right side facing out. Using the smaller circular needles and beginning at the left shoulder")
    print("seam, pick up and knit 15 stitches along the left front neck edge, 29 stitches across the center front neck edge,")
    print("15 stitches along the right front neck edge, then knit the 41 stitches off of the back stitch holder. --- 100 stitches.")
    print( )
    print("Place marker and join for working in the round.")
    print( )
    if collar == '1x1 rib':
        print("Ribbing: *k1, p1 repeat from * around.")
        print( )
        print("Work in 1x1 ribbing until length of ribbing equals 1.5 inches, then use a stretchy bind off to finish.")
    elif collar == 'Folded':
        print("Ribbing: *k1, p1 repeat from * around.")
        print( )
        print("Work in 1x1 ribbing until length of ribbing equals 4 inches.")
        print( )
        print("Once you have 4 inches of ribbing, maneuver your work so that the Wrong Side is flush with the Right Side that")
        print("you are knitting on. Next, pick up and knit the corresponding bottom stitch (from your join).")
        print("This is your next row. Instead of knitting just your 1x1 ribbing stitches you just made, you are knitting those")
        print("stitches PLUS your first row stitches together. This forms the 'fold' of your folded collar.")
        print("Purl one more row across.")
        print( )
        print("Bind off")
    else:
        print("Knit every round until collar measures 2.5 inches.")
        print( )
        print("Bind off")
    print( )
    print("Sew in sleeves.")
    print( )
    print("Sew side and sleeve seams.")
    print( )
    print("Weave in all ends and block as desired.")
          
collar_options = ['1x1 rib', 'Rolled', 'Folded']
collar_dropdown = widgets.Dropdown(
    options=collar_options,
    value=collar_options[0],
    description='Collar:',
)

cuffs_options = ['1x1 rib', 'Rolled', 'Folded']
cuffs_dropdown = widgets.Dropdown(
    options=cuffs_options,
    value=cuffs_options[0],
    description='Cuffs:',
)

chest_circumference_input = widgets.FloatText(
    value=43,
    min=24.0,
    max=67.0,
    step=0.1,
    description='Chest Circumference:',
    disabled=False
)

torso_length_input = widgets.FloatText(
    value=23.5,
    min=16.0,
    max=29.0,
    step=0.1,
    description='Torso Length:',
    disabled=False
)

interact_manual(run_knitting_design, 
                collar=collar_dropdown, 
                cuffs=cuffs_dropdown, 
                chest_circumference=chest_circumference_input, 
                torso_length=torso_length_input)

interactive(children=(Dropdown(description='Collar:', options=('1x1 rib', 'Rolled', 'Folded'), value='1x1 rib'…

<function __main__.run_knitting_design(collar, cuffs, chest_circumference, torso_length)>

# Related Internet Resources

1. This site gave instructions for how to code my interactive widget and it was the most important to me for this project [Using Interact](https://ipywidgets.readthedocs.io/en/8.1.5/examples/Using%20Interact.html).
2. I also frequently (Constantly) referenced [Python Tutorial](https://docs.python.org/3/tutorial/index.html)
3. Here is a site that contains documentation for our main [blah blah package](https://us.prairielearn.com/pl/course_instance/137200/instructor/question/9076129/preview)
4. [This site](https://www.stitchfiddle.com/en) Calls itself a knitting pattern generator, but it only allows users to make pixel-art style grid charts for colorwork. My program produces a written pattern.